In [50]:
import google.generativeai as genai
import pandas as pd
import time
import numpy as np
from scipy import stats
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [51]:
cleaned_df = pd.read_csv("data/cleaned_dataset/merged.tsv", sep=",", encoding="utf-8")

In [52]:
sampled_df = cleaned_df.groupby("Party_orientation", group_keys=False).apply(lambda x: x.sample(n=30, random_state=42))

C:\Users\fabio\AppData\Local\Temp\ipykernel_15060\1887085074.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = cleaned_df.groupby("Party_orientation", group_keys=False).apply(lambda x: x.sample(n=30, random_state=42))


In [53]:
for elem in set(sampled_df["Party_orientation"]):
    print(elem, sampled_df[sampled_df["Party_orientation"]==elem].shape[0])

Destra verso estrema destra 30
Centro-sinistra 30
Centro verso centro-destra 30
Centro 30
Pigliatutto 30
Centro-destra 30


In [54]:
#Estraggo i testi da dare in input a gemini
texts_to_classify = []
for id in sampled_df["ID"]:
    texts_to_classify.append((id, sampled_df[sampled_df["ID"] == id].text.iloc[0]))

In [ ]:
API_KEY = ""
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash')

In [56]:
def classify_record(text_id, text, prompt, model):
    filled_prompt = prompt.replace('[$$$]',text)
    res = model.generate_content(filled_prompt)
    time.sleep(4)
    return str(text_id + "," +res.text)

In [57]:
#CALCOLO CLASSIFICAZIONI GEMINI
def calcola_classificazioni(texts_to_classify, prompt):
    i = 0
    gemini_classes = []
    for tupla in texts_to_classify:
        res = classify_record(tupla[0], tupla[1], prompt, model)
        gemini_classes.append(res)
        i+=1
        print(i, end = " ")
        
    classification_list = []
    for stringa in gemini_classes:
        stringa = stringa.strip("\n")
        lista = stringa.split(",")
        lista = [lista[0]] + [int(x) for x in lista[1:]]
        classification_list.append(lista)

    classification_df = pd.DataFrame(classification_list, columns = ['id', 'polarization'])
    return classification_df

In [58]:
def calcola_prestazioni(ground_df, classification_df):
    dimensions = ['polarization']
    for dim in dimensions:
        print("ANALISI: ", dim)
        print("SPEARMAN: ", stats.spearmanr(ground_df[dim], classification_df[dim]))
        print("MSE: ", mean_squared_error(ground_df[dim], classification_df[dim]))
        print("MAE: ", mean_absolute_error(ground_df[dim], classification_df[dim]))
        print(classification_report(ground_df[dim], classification_df[dim]))
        print("\n\n")

In [59]:
diz = {
    "Pigliatutto": 2,
    "Centro": 2,
    "Centro-sinistra": 1,
    "Centro verso centro-destra": 3,
    "Centro-destra": 3,
    "Destra verso estrema destra": 3
}

In [60]:
sampled_df = sampled_df.replace({"Party_orientation": diz})
sampled_df["polarization"] = sampled_df["Party_orientation"]

C:\Users\fabio\AppData\Local\Temp\ipykernel_15060\544985932.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sampled_df = sampled_df.replace({"Party_orientation": diz})


In [42]:
with open('genai/v2/zero_shot_auto_prompt.txt', 'r', encoding="utf8") as f:
    zero_shot_auto_prompt = f.read()

In [45]:
zero_shot_auto_classification = calcola_classificazioni(texts_to_classify, zero_shot_auto_prompt)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 

In [48]:
zero_shot_auto_classification.to_csv("genai/v2/zero_shot_auto_classification.csv", index=False)

In [49]:
calcola_prestazioni(sampled_df, zero_shot_auto_classification)

ANALISI:  polarization
SPEARMAN:  SignificanceResult(statistic=0.6553563264112805, pvalue=1.8356698804330645e-23)
MSE:  0.7
MAE:  0.5555555555555556
              precision    recall  f1-score   support

           1       0.39      0.43      0.41        30
           2       0.42      0.38      0.40        60
           3       0.69      0.58      0.63        60
           4       0.54      0.73      0.62        30

    accuracy                           0.52       180
   macro avg       0.51      0.53      0.52       180
weighted avg       0.52      0.52      0.52       180




